# **Selenium Python Framework Implementation**

- Standard of writing selenium tests in framework
- Creating browser invocation Fixtures in confest.py
- Setting up base class to hold all common Utilities
- Inheriting Base class to all tests to remove fixture redundant Code
- Passing command line options to select browser at run time
- Implementing page object mechanism
- Smarter way of returning page objects from navigation methods
- Creating selenium webdriver custom utilities in base class
- Parameterising webdriver tests with multiple data sets
- Organizing data from separate data files and injecting into fixture at run time
- Implementing logging feature to webdriver tests 
- Text execution HTML reporting
- Automatic screenshot capture on test failures
- Integrating selenium python framework to jenkins CI tool with jenkin build Parameterization
- Github basics for project version control


To create a robust **Selenium Python Test Automation Framework** using **Pytest**, **Jenkins**, and **GitHub**, follow this step-by-step guide that addresses all the points you mentioned. This will help you build a maintainable, scalable, and efficient automation framework.

## **1. Setting Up the Project Structure**

Start by setting up a proper folder structure for your Selenium-Pytest framework:

```
selenium_pytest_framework/
│
├── tests/                   # Holds all test files
│   ├── test_sample.py
│
├── pages/                   # Holds page object model files
│   ├── base_page.py
│   ├── login_page.py
│
├── utils/                   # Custom utilities (common methods, logging)
│   ├── logger.py
│   ├── data_utils.py
│
├── data/                    # Data files for parameterized tests
│   ├── test_data.json
│
├── reports/                 # Test reports and screenshots
│
├── screenshots/             # Stores screenshots on failures
│
├── conftest.py              # Fixtures and setup code
│
├── requirements.txt         # All project dependencies
│
├── pytest.ini               # Pytest configuration
│
├── Jenkinsfile              # Jenkins build file
│
└── README.md                # Project documentation
```

## **2. Standard of Writing Selenium Tests in Framework**

In each test file, follow a structured format where:
- You follow the **Arrange-Act-Assert** pattern.
- Use **Page Object Model (POM)** for managing locators and actions for each page.

```python
from pages.login_page import LoginPage

def test_login_valid_user(init_browser):
    login_page = LoginPage(init_browser)
    
    # Arrange
    login_page.navigate_to_login()
    
    # Act
    login_page.login('user', 'password')

    # Assert
    assert login_page.is_login_successful()
```

## **3. Browser Invocation Fixtures in `conftest.py`**

You need to use **pytest fixtures** in `conftest.py` to initialize the WebDriver and clean up after test execution.

```python
import pytest
from selenium import webdriver

@pytest.fixture(params=["chrome", "firefox"], scope='class')
def init_browser(request):
    browser = request.param
    if browser == "chrome":
        driver = webdriver.Chrome()
    elif browser == "firefox":
        driver = webdriver.Firefox()
    driver.maximize_window()
    driver.implicitly_wait(10)
    
    request.cls.driver = driver
    yield driver
    driver.quit()
```
In the provided `pytest` fixture, `request` is an object of the `pytest.FixtureRequest` class, which is passed as an argument to the fixture function. It provides access to information about the test function or class that is currently being executed. Here's how `request` is used in your code:

### Key Uses of `request` in the Fixture:

1. **Accessing the `param` attribute**:
   - `request.param` allows access to the parameters that are passed to the fixture. In this case, the fixture is parameterized with `params=["chrome", "firefox"]`, which means `request.param` will contain either `"chrome"` or `"firefox"`, depending on which test is being run.
   
   Example:
   ```python
   browser = request.param  # 'chrome' or 'firefox' based on the test run
   ```

2. **Setting the `driver` attribute on the class (`request.cls`)**:
   - `request.cls.driver = driver` assigns the `driver` object (the WebDriver instance) to the test class. This allows the WebDriver to be accessible to all test methods within the class where this fixture is used.

   Example:
   ```python
   request.cls.driver = driver  # Makes the driver accessible to the test class
   ```

### Breakdown of the Code:
- `@pytest.fixture`: Marks the function as a fixture, and it can be shared across multiple test functions or classes.
- `params=["chrome", "firefox"]`: The fixture will run twice for each test that uses it, once with `"chrome"` and once with `"firefox"`.
- `request.param`: Refers to the browser type, which is either `"chrome"` or `"firefox"`.
- `request.cls`: Refers to the test class that is using this fixture. By assigning `driver` to `request.cls.driver`, the WebDriver is made available in the test class as `self.driver`.

So, `request` helps in:
- Providing the parameter values passed to the fixture.
- Interacting with the test class to inject or share resources like the WebDriver across test methods.

## **4. Base Class for Common Utilities**

The **base class** holds common methods that can be reused across different test cases, such as navigation, waits, screenshot capture, etc.

```python
class BasePage:
    def __init__(self, driver):
        self.driver = driver

    def capture_screenshot(self, name):
        self.driver.save_screenshot(f"./screenshots/{name}.png")

    def wait_for_element(self, by, value, timeout=10):
        WebDriverWait(self.driver, timeout).until(EC.presence_of_element_located((by, value)))
```

## **5. Inherit Base Class in All Tests**

In each test class, inherit the `BasePage` to avoid redundant code in your test files:

```python
class LoginPage(BasePage):
    def __init__(self, driver):
        super().__init__(driver)

    def navigate_to_login(self):
        self.driver.get("https://example.com/login")

    def login(self, username, password):
        self.driver.find_element(By.ID, "user").send_keys(username)
        self.driver.find_element(By.ID, "pass").send_keys(password)
        self.driver.find_element(By.ID, "login").click()

    def is_login_successful(self):
        return "Dashboard" in self.driver.title
```

## **6. Command Line Options for Browser Selection**

In `pytest`, command-line options can be used to pass configuration or values to your tests dynamically, such as selecting the browser to run your Selenium tests on. You can achieve this by defining a custom command-line option and accessing it within your tests or fixtures. Here's how you can implement browser selection via command-line options.

### Steps to Implement Command-Line Browser Selection in `pytest`:

1. **Add a Custom Command-Line Option**:
   You can use the `pytest_addoption` hook to add custom options. This hook should be placed inside a `conftest.py` file, which is a configuration file for `pytest` that can define fixtures, hooks, and custom command-line options.

2. **Access the Command-Line Option in Your Fixture**:
   After defining the custom command-line option, you can access it in your fixture to decide which browser to use for the WebDriver.

### Example Code:

#### 1. Define a Custom Command-Line Option in `conftest.py`:

```python
import pytest

# Adding a custom command-line option to select the browser
def pytest_addoption(parser):
    parser.addoption(
        "--browser", 
        action="store", 
        default="chrome",  # Default browser is Chrome
        help="Browser option: chrome or firefox"
    )
```

This defines a command-line option `--browser`, with a default value of `"chrome"`. You can pass `"firefox"` as a value when running your tests.

#### 2. Access the Option in Your Fixture:

```python
import pytest
from selenium import webdriver

@pytest.fixture(scope="class")
def init_browser(request):
    # Accessing the browser option from the command line
    browser = request.config.getoption("--browser")
    
    if browser == "chrome":
        driver = webdriver.Chrome()
    elif browser == "firefox":
        driver = webdriver.Firefox()
    else:
        raise ValueError(f"Unsupported browser: {browser}")
    
    driver.maximize_window()
    driver.implicitly_wait(10)
    request.cls.driver = driver
    yield driver
    driver.quit()
```

In this example:
- `request.config.getoption("--browser")`: Fetches the value of the `--browser` command-line option.
- Based on the value (either `"chrome"` or `"firefox"`), the WebDriver for the corresponding browser is instantiated.
- The WebDriver is accessible within the test class as `self.driver`.

#### 3. Example Test Case:

```python
import pytest

@pytest.mark.usefixtures("init_browser")
class TestSample:
    def test_open_url(self):
        self.driver.get("https://www.google.com")
        assert "Google" in self.driver.title
```

### Running the Tests with Command-Line Options:

You can now run the tests by specifying the browser on the command line.

- To run with Chrome (default):
  ```bash
  pytest
  ```

- To run with Firefox:
  ```bash
  pytest --browser=firefox
  ```

### How It Works:
- By adding the `--browser` command-line option, you can select which browser to use when running your tests.
- The default browser is set to Chrome, but you can override it with `--browser=firefox` or any other supported browser.
  
### Benefits of Using Command-Line Browser Selection:
- **Flexibility**: Allows you to run the same test suite on multiple browsers without changing the test code.
- **Scalability**: Useful in Continuous Integration (CI) pipelines where tests need to run across different browsers.
- **Maintainability**: Keeps the test configuration clean and avoids hardcoding browser settings in the test code.

This approach enhances the usability and flexibility of your test framework by letting you choose the browser at runtime.

## **6.1. Mobile View**

Yes, you can configure Selenium WebDriver to simulate mobile view by setting the browser in mobile emulation mode. For Chrome and Firefox, this can be done by setting specific options. Below are the steps to achieve this for both Chrome and Firefox.

### 1. **Mobile Emulation in Chrome**

In Chrome, you can use the `ChromeOptions` class to set the mobile device emulation using a predefined device or by specifying custom width, height, and user-agent.

#### Example Code for Mobile Emulation in Chrome:

```python
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

def init_mobile_browser():
    # Set up ChromeOptions for mobile emulation
    mobile_emulation = {
        "deviceName": "iPhone X"  # You can specify any predefined device
    }
    
    chrome_options = Options()
    chrome_options.add_experimental_option("mobileEmulation", mobile_emulation)

    driver = webdriver.Chrome(options=chrome_options)
    driver.maximize_window()
    return driver

# Example usage
driver = init_mobile_browser()
driver.get("https://www.google.com")
print(driver.title)
driver.quit()
```

#### Specifying Custom Mobile Screen Dimensions:
If you don’t want to use a predefined device, you can specify custom dimensions for screen size and other parameters like the user agent.

```python
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

def init_custom_mobile_browser():
    # Custom mobile emulation settings
    mobile_emulation = {
        "deviceMetrics": { "width": 360, "height": 640, "pixelRatio": 3.0 },
        "userAgent": "Mozilla/5.0 (iPhone; CPU iPhone OS 13_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0 Mobile/15A372 Safari/604.1"
    }
    
    chrome_options = Options()
    chrome_options.add_experimental_option("mobileEmulation", mobile_emulation)

    driver = webdriver.Chrome(options=chrome_options)
    return driver

# Example usage
driver = init_custom_mobile_browser()
driver.get("https://www.google.com")
print(driver.title)
driver.quit()
```

In the above code:
- `"deviceName": "iPhone X"`: Simulates iPhone X device. You can replace it with other devices like `"Pixel 2"`, `"Nexus 5"`, etc.
- `"deviceMetrics"`: Manually specifies the width, height, and pixel ratio for the custom device.
- `"userAgent"`: Defines the user agent string to mimic a mobile browser.

### 2. **Mobile View in Firefox**

For Firefox, you can use the `FirefoxOptions` class to specify the size of the browser window to simulate mobile dimensions. Firefox doesn't have a built-in mobile emulation feature like Chrome, but you can manually set the window size.

#### Example Code for Mobile View in Firefox:

```python
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

def init_firefox_mobile_view():
    options = Options()
    driver = webdriver.Firefox(options=options)
    
    # Set the browser to a mobile-like window size
    driver.set_window_size(360, 640)  # Width, Height for a mobile screen
    return driver

# Example usage
driver = init_firefox_mobile_view()
driver.get("https://www.google.com")
print(driver.title)
driver.quit()
```

In the above code:
- `driver.set_window_size(360, 640)`: Manually sets the browser's window size to mobile dimensions.

### 3. **Running Tests in Mobile View Using Pytest**

You can integrate this setup into a `pytest` framework by adding mobile browser options dynamically based on the command-line options.

#### Example `pytest` Fixture with Mobile Browser Options:

```python
import pytest
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

@pytest.fixture(scope="class")
def init_browser(request):
    browser = request.config.getoption("--browser")
    mobile = request.config.getoption("--mobile")
    
    if browser == "chrome":
        chrome_options = Options()
        
        # Check if mobile option is passed
        if mobile:
            mobile_emulation = {"deviceName": "iPhone X"}  # Simulate iPhone X
            chrome_options.add_experimental_option("mobileEmulation", mobile_emulation)
        
        driver = webdriver.Chrome(options=chrome_options)
    elif browser == "firefox":
        driver = webdriver.Firefox()
        if mobile:
            driver.set_window_size(360, 640)  # Set Firefox to mobile-like dimensions
    else:
        raise ValueError("Unsupported browser!")

    driver.maximize_window()
    request.cls.driver = driver
    yield driver
    driver.quit()

def pytest_addoption(parser):
    parser.addoption("--browser", action="store", default="chrome", help="Browser option: chrome or firefox")
    parser.addoption("--mobile", action="store_true", help="Enable mobile emulation")

# Example test
@pytest.mark.usefixtures("init_browser")
class TestMobileView:
    def test_open_mobile_view(self):
        self.driver.get("https://www.google.com")
        assert "Google" in self.driver.title
```

#### Running the Tests:
- Run with Chrome (desktop):
  ```bash
  pytest --browser=chrome
  ```
- Run with Chrome in mobile emulation mode:
  ```bash
  pytest --browser=chrome --mobile
  ```
- Run with Firefox in mobile view:
  ```bash
  pytest --browser=firefox --mobile
  ```

### Conclusion

By using Chrome's mobile emulation and manually resizing the window in Firefox, you can simulate a mobile device environment for your Selenium tests. This is useful for testing the responsiveness of websites and ensuring a consistent user experience across different devices.


## **7. Page Object Mechanism**

Follow the **Page Object Model** (POM) for separating UI actions and elements into dedicated classes for each page of your application. This improves maintainability.

An optimized Page Object Model (POM) structure can significantly improve the efficiency and maintainability of your test automation. Here are some advanced ways to enhance the traditional POM beyond method chaining and object return:

### 1. **Single Responsibility Principle (SRP) with Separate Actions and Validations**
   - Keep actions and validations separate within page objects. This separation helps maintain clarity between the functionality a page offers and the assertions made on it.
   - The actions focus on interacting with the page, while validations or assertions can be kept in dedicated methods or even separate classes.

### Example:

```python
class LoginPage:
    def __init__(self, driver):
        self.driver = driver

    def enter_username(self, username):
        self.driver.find_element_by_id("username").send_keys(username)

    def enter_password(self, password):
        self.driver.find_element_by_id("password").send_keys(password)

    def click_login(self):
        self.driver.find_element_by_id("login").click()

class LoginPageAssertions:
    def __init__(self, driver):
        self.driver = driver

    def verify_login_failed(self):
        return "Invalid credentials" in self.driver.page_source
```

### 2. **Page Factory (Lazy Initialization)**
The **PageFactory** pattern is an enhancement to the POM approach that initializes web elements only when they are accessed (lazy loading). This avoids unnecessary element lookups and improves efficiency when dealing with dynamic or large pages.

- **Benefit:** Improves test performance by delaying the initialization of elements until they're needed.

### Example:

```python
from selenium.webdriver.support.page_factory import PageFactory
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

class LoginPage:
    def __init__(self, driver):
        self.driver = driver
        PageFactory.init_elements(driver, self)  # Lazy initialization

    @property
    def username_field(self):
        return WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.ID, "username"))
        )

    @property
    def password_field(self):
        return self.driver.find_element(By.ID, "password")

    @property
    def login_button(self):
        return self.driver.find_element(By.ID, "login")

    def login(self, username, password):
        self.username_field.send_keys(username)
        self.password_field.send_keys(password)
        self.login_button.click()
```

### 3. **Service Layer for Business Logic**
Instead of tightly coupling your test logic with page actions, you can abstract **business workflows** in a service layer. This further decouples the test from UI details and focuses on the business scenario. The service layer handles page-to-page navigation and business workflows, allowing reuse of logic across multiple tests.

### Example:

```python
class LoginService:
    def __init__(self, driver):
        self.driver = driver
        self.login_page = LoginPage(driver)
        self.home_page = HomePage(driver)

    def login_as(self, username, password):
        self.login_page.enter_username(username)
        self.login_page.enter_password(password)
        self.login_page.click_login()
        return self.home_page

class TestLogin:
    def test_successful_login(self):
        login_service = LoginService(self.driver)
        home_page = login_service.login_as("testuser", "password123")
        assert home_page.is_welcome_message_displayed()
```

### 4. **Fluent Interface**
The **Fluent Interface Pattern** allows chaining of method calls for readability and fluidity in interaction. This pattern is especially useful for tests that involve multiple steps on a page. Fluent interfaces allow you to avoid excessive object creation and allow easy chaining of methods with assertions.

### Example:

```python
class LoginPage:
    def __init__(self, driver):
        self.driver = driver

    def enter_username(self, username):
        self.driver.find_element_by_id("username").send_keys(username)
        return self  # Returning the object itself for method chaining

    def enter_password(self, password):
        self.driver.find_element_by_id("password").send_keys(password)
        return self

    def click_login(self):
        self.driver.find_element_by_id("login").click()
        return HomePage(self.driver)

class TestLogin:
    def test_valid_login(self):
        home_page = (LoginPage(self.driver)
                     .enter_username("user123")
                     .enter_password("password123")
                     .click_login())
        assert home_page.is_dashboard_visible()
```

### 5. **Component Object Model**
For complex applications, a **Component Object Model** can be a useful extension of the POM. This involves creating reusable components (or fragments) of a page, such as navigation bars, side menus, or footers, which can be reused across different page objects.

### Example:

```python
class NavBarComponent:
    def __init__(self, driver):
        self.driver = driver

    def click_home(self):
        self.driver.find_element_by_id("home").click()
        return HomePage(self.driver)

    def click_settings(self):
        self.driver.find_element_by_id("settings").click()
        return SettingsPage(self.driver)

class HomePage:
    def __init__(self, driver):
        self.driver = driver
        self.navbar = NavBarComponent(driver)  # Reuse the component in multiple pages

class SettingsPage:
    def __init__(self, driver):
        self.driver = driver
        self.navbar = NavBarComponent(driver)
```

### 6. **Factory Pattern for Page Object Creation**
In this approach, a factory class handles the creation of page objects dynamically based on the current URL or other criteria. This reduces boilerplate code in tests and ensures that the correct page object is always used.

### Example:

```python
class PageFactory:
    @staticmethod
    def get_page(driver):
        current_url = driver.current_url
        if "login" in current_url:
            return LoginPage(driver)
        elif "home" in current_url:
            return HomePage(driver)
        # Add more conditions for other pages
        return BasePage(driver)
```

### 7. **Centralizing WebDriver Utilities**
Create a base class or utility class where all WebDriver-related methods (clicking elements, sending keys, waiting for elements) are centralized. This avoids code duplication and makes debugging easier.

### Example:

```python
class BasePage:
    def __init__(self, driver):
        self.driver = driver

    def wait_for_element(self, locator):
        WebDriverWait(self.driver, 10).until(EC.presence_of_element_located(locator))

    def click(self, locator):
        self.wait_for_element(locator)
        self.driver.find_element(locator).click()

class LoginPage(BasePage):
    def login(self, username, password):
        self.click((By.ID, "login_button"))
```

### 8. **Decoupling Tests and Page Objects with Dependency Injection**
Instead of tightly coupling tests and page objects, use dependency injection frameworks (such as `pytest fixtures`) to inject page objects into test cases dynamically. This improves reusability and reduces redundancy.

### Example with `pytest` fixtures:

```python
@pytest.fixture
def login_page(driver):
    return LoginPage(driver)

def test_login(login_page):
    home_page = login_page.login_as("user", "password")
    assert home_page.is_loaded()
```

### Conclusion:
These are some of the advanced ways to improve the **Page Object Model**:
- **Separate Actions and Assertions:** Helps in maintaining a cleaner separation of concerns.
- **Lazy Initialization (Page Factory):** Improves efficiency and handles dynamic elements well.
- **Service Layer:** Encapsulates business logic, reducing the coupling between test and page layers.
- **Fluent Interface Pattern:** Improves readability and allows method chaining.
- **Component Model:** Encourages reusability of common page components across different pages.
- **Factory Pattern:** Simplifies object creation logic and reduces redundancy.
- **Centralized WebDriver Utilities:** Makes test maintenance easier by reducing code duplication.

Each method brings its own set of advantages, and depending on the complexity of your test suite, you can implement a combination of these patterns to maximize the efficiency and readability of your Selenium tests.


## **8. Returning Page Objects from Navigation Methods**

Use method chaining to navigate through pages and return page objects for smooth transitions:

```python
def login(self, username, password):
    self.driver.find_element(By.ID, "user").send_keys(username)
    self.driver.find_element(By.ID, "pass").send_keys(password)
    self.driver.find_element(By.ID, "login").click()
    return DashboardPage(self.driver)
```

## **9. Custom Selenium WebDriver Utilities**

Add custom WebDriver utilities such as scrolling, taking screenshots, or waiting for elements in the **BasePage** class for reuse.

## **10. Parameterizing WebDriver Tests with Multiple Data Sets**

Use **pytest.mark.parametrize** to provide multiple data sets to a single test:

```python
@pytest.mark.parametrize("username, password", [
    ("user1", "password1"),
    ("user2", "password2")
])
def test_login(init_browser, username, password):
    login_page = LoginPage(init_browser)
    login_page.login(username, password)
    assert login_page.is_login_successful()
```

## **11. Organize Data from Separate Files**

Store test data in separate files like JSON or CSV and inject the data into the test fixture dynamically:

```python
import json

@pytest.fixture(params=json.load(open('data/test_data.json')))
def get_data(request):
    return request.param
```

## **12. Implement Logging Feature**

Create a logging utility using Python’s `logging` module and integrate it into your framework to track test execution.

```python
import logging

def get_logger():
    logger = logging.getLogger(__name__)
    file_handler = logging.FileHandler('test.log')
    formatter = logging.Formatter('%(asctime)s : %(levelname)s : %(message)s')
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)
    logger.setLevel(logging.INFO)
    return logger
```

## **13. HTML Reporting with Pytest**

Use **pytest-html** for generating HTML reports after test execution.

Install the plugin:
```bash
pip install pytest-html
```

Run tests with HTML report generation:
```bash
pytest --html=reports/test_report.html
```

## **14. Automatic Screenshot Capture on Test Failures**

Add a hook in `conftest.py` to capture screenshots automatically when a test fails:

```python
@pytest.hookimpl(tryfirst=True, hookwrapper=True)
def pytest_runtest_makereport(item, call):
    outcome = yield
    report = outcome.get_result()
    if report.when == 'call' and report.failed:
        driver = item.instance.driver
        driver.save_screenshot(f"screenshots/{item.name}.png")
```

## **15. Jenkins Integration with Build Parameterization**

1. **Install Jenkins** and add the **Pytest plugin**.
2. Create a **Jenkinsfile** for pipeline execution:
   ```groovy
   pipeline {
       agent any
       stages {
           stage('Test') {
               steps {
                   sh 'pytest --html=reports/test_report.html'
               }
           }
       }
   }
   ```
3. Use Jenkins **build parameters** to run tests on different browsers or environments.

## **16. GitHub for Version Control**

1. Initialize a Git repository:
   ```bash
   git init
   git add .
   git commit -m "Initial commit"
   ```
2. Push the code to GitHub:
   ```bash
   git remote add origin <remote_url>
   git push -u origin master
   ```

This guide provides a robust framework with **Selenium, Pytest, Page Object Model, Jenkins**, and **GitHub** integration. It also includes parameterization, logging, HTML reporting, and automatic screenshot capture on test failures.

---